# Gesture Recognition
In this group project, you are going to build a 3D Conv model that will be able to predict the 5 gestures correctly. Please import the following libraries to get started.

In [1]:
import numpy as np
import os
from scipy.misc import imread, imresize
import datetime
import os
import cv2

In [2]:
from platform import python_version
print(python_version())
import scipy
print(scipy.__version__)

3.6.8
1.1.0


We set the random seed so that the results don't vary drastically.

In [3]:
np.random.seed(30)
import random as rn
rn.seed(30)

import keras
from keras import backend as K
print('keras version:', keras.__version__)

import tensorflow as tf
print('tensorflow version:', tf.__version__)
tf.set_random_seed(30)
#tf.random.set_seed(30)

Using TensorFlow backend.


keras version: 2.2.2
tensorflow version: 1.10.0


config = tf.ConfigProto()
config.gpu_options.allow_growth = True 
config.log_device_placement = True
sess = tf.Session(config=config)
tf.keras.backend.set_session(sess)

In this block, you read the folder names for training and validation. You also set the `batch_size` here. Note that you set the batch size in such a way that you are able to use the GPU in full capacity. You keep increasing the batch size until the machine throws an error.

train_doc = np.random.permutation(open('/Users/sarath_mac/Downloads/upGrad/assignment_neural/Project_data/train.csv').readlines())
val_doc = np.random.permutation(open('/Users/sarath_mac/Downloads/upGrad/assignment_neural/Project_data/val.csv').readlines())
batch_size = 32

In [4]:
train_doc = np.random.permutation(open('./Project_data/train.csv').readlines())
val_doc = np.random.permutation(open('./Project_data/val.csv').readlines())
batch_size = 32 #experiment with the batch size

In [5]:
#Declare Variables 
input_rows=32
input_cols=32
input_channels=3
num_images=16
Num_Classes=5
batch_inp_shape=(batch_size,num_images,input_rows,input_cols,input_channels)

## Generator
This is one of the most important part of the code. The overall structure of the generator has been given. In the generator, you are going to preprocess the images as you have images of 2 different dimensions as well as create a batch of video frames. You have to experiment with `img_idx`, `y`,`z` and normalization such that you get high accuracy.

In [6]:
def generator(source_path, folder_list, batch_size):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    img_idx = [i*2 for i in range(15)] #create a list of image numbers you want to use for a particular video
    while True:
        t = np.random.permutation(folder_list)
        num_batches = len(folder_list)//batch_size # calculate the number of batches
        for batch in range(num_batches): # we iterate over the number of batches
#            print('batch: ',batch)
            batch_data = np.zeros((batch_size,num_images,input_rows,input_cols,input_channels)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,Num_Classes)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    img=imresize(image,(input_rows,input_cols,input_channels))
                    norm_image = cv2.normalize(img, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
                    batch_data[folder,idx,:,:,0] = norm_image[:,:,0] #normalise and feed in the image
                    
                    batch_data[folder,idx,:,:,1] = norm_image[:,:,1] #normalise and feed in the image
                    
                    batch_data[folder,idx,:,:,2] = norm_image[:,:,2] #normalise and feed in the image
                    
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do

        
        # write the code for the remaining data points which are left after full batches
        if (len(folder_list)%batch_size) !=0:
 #           print('batch=20')
            imgs = os.listdir(source_path+'/'+t[((len(folder_list)%batch_size)-1) + ((len(folder_list)//batch_size)*batch_size)].split(';')[0]) # read all the images in the folder
            for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                image = imread(source_path+'/'+ t[((len(folder_list)%batch_size)-1) + ((len(folder_list)//batch_size)*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    
                #crop the images and resize them. Note that the images are of 2 different shape 
                #and the conv3D will throw error if the inputs in a batch have different shapes
                img=imresize(image,(input_rows,input_cols,input_channels))
                norm_image = cv2.normalize(img, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
                batch_data[folder,idx,:,:,0] = norm_image[:,:,0] #normalise and feed in the image
                  
                batch_data[folder,idx,:,:,1] = norm_image[:,:,1] #normalise and feed in the image
                
                batch_data[folder,idx,:,:,2] = norm_image[:,:,2] #normalise and feed in the image
                    
            batch_labels[folder, int(t[((len(folder_list)%batch_size)-1) + ((len(folder_list)//batch_size)*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do

Note here that a video is represented above in the generator as (number of images, height, width, number of channels). Take this into consideration while creating the model architecture.

curr_dt_time = datetime.datetime.now()
train_path = '/Users/sarath_mac/Downloads/upGrad/assignment_neural/train'
val_path = '/Users/sarath_mac/Downloads/upGrad/assignment_neural/val'
num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)
num_epochs = 5
print ('# epochs =', num_epochs)

In [7]:
curr_dt_time = datetime.datetime.now()
train_path = "./Project_data/train"
val_path = "./Project_data/val"
num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)
num_epochs = 50 # choose the number of epochs
print ('# epochs =', num_epochs)

# training sequences = 663
# validation sequences = 100
# epochs = 50


## Model
Here you make the model using different functionalities that Keras provides. Remember to use `Conv3D` and `MaxPooling3D` and not `Conv2D` and `Maxpooling2D` for a 3D convolution model. You would want to use `TimeDistributed` while building a Conv2D + RNN model. Also remember that the last layer is the softmax. Design the network in such a way that the model is able to give good accuracy on the least number of parameters so that it can fit in the memory of the webcam.

# CONV2D +RNN

In [8]:
from keras.models import Sequential, Model
from keras.layers import Dense, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras import optimizers

#write your model here
#covolution layers
conv2d_model = Sequential()
conv2d_model.add(TimeDistributed(Conv2D(32,kernel_size=(3,3),activation='relu',padding='same',data_format="channels_last",batch_input_shape=batch_inp_shape)))
conv2d_model.add(TimeDistributed(BatchNormalization()))

conv2d_model.add(TimeDistributed(Conv2D(64,kernel_size=(3,3),activation='relu')))
conv2d_model.add(TimeDistributed(BatchNormalization()))
conv2d_model.add(TimeDistributed(MaxPooling2D(pool_size=(2,2))))

# extract features
conv2d_model.add(TimeDistributed(Flatten()))

#RNN layers
conv2d_model.add(GRU(8, return_sequences=True))
conv2d_model.add(Flatten())
conv2d_model.add(Dense(32,activation='relu'))
conv2d_model.add(Dense(Num_Classes,activation='softmax'))

Now that you have written the model, the next step is to `compile` the model. When you print the `summary` of the model, you'll see the total number of parameters you have to train.

In [9]:
conv2d_model.build(batch_inp_shape)
conv2d_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_1 (TimeDist (32, 16, 32, 32, 32)      896       
_________________________________________________________________
time_distributed_2 (TimeDist (32, 16, 32, 32, 32)      128       
_________________________________________________________________
time_distributed_3 (TimeDist (32, 16, 30, 30, 64)      18496     
_________________________________________________________________
time_distributed_4 (TimeDist (32, 16, 30, 30, 64)      256       
_________________________________________________________________
time_distributed_5 (TimeDist (32, 16, 15, 15, 64)      0         
_________________________________________________________________
time_distributed_6 (TimeDist (32, 16, 14400)           0         
_________________________________________________________________
gru_1 (GRU)                  (32, 16, 8)               345816    
__________

In [10]:
optimiser = optimizers.Adam() #write your optimizer
conv2d_model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])

Let us create the `train_generator` and the `val_generator` which will be used in `.fit_generator`.

In [11]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [12]:
model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, cooldown=1, verbose=1) # write the REducelronplateau code here
callbacks_list = [checkpoint, LR]

The `steps_per_epoch` and `validation_steps` are used by `fit_generator` to decide the number of next() calls it need to make.

In [13]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

Let us now fit the model. This will start training the model and with the help of the checkpoints, you'll be able to save the model at the end of each epoch.

In [14]:
conv2d_model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path = Source path =  ./Project_data/train ; batch size = 32
Epoch 1/50
 ./Project_data/val ; batch size = 32


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:14: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:18: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


 3/21 [===>..........................] - ETA: 39s - loss: 1.6430 - categorical_accuracy: 0.2500 

/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:35: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:39: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


21/21 [==============================] - 36s 2s/step - loss: 1.5825 - categorical_accuracy: 0.2738 - val_loss: 1.5075 - val_categorical_accuracy: 0.3906

Epoch 00001: saving model to model_init_2020-05-2414_09_58.387065/model-00001-1.58254-0.27381-1.50746-0.39062.h5
Epoch 2/50
21/21 [==============================] - 31s 1s/step - loss: 1.3203 - categorical_accuracy: 0.4955 - val_loss: 1.3677 - val_categorical_accuracy: 0.4219

Epoch 00002: saving model to model_init_2020-05-2414_09_58.387065/model-00002-1.32028-0.49554-1.36775-0.42188.h5
Epoch 3/50
21/21 [==============================] - 32s 2s/step - loss: 1.1500 - categorical_accuracy: 0.5372 - val_loss: 1.2705 - val_categorical_accuracy: 0.5156

Epoch 00003: saving model to model_init_2020-05-2414_09_58.387065/model-00003-1.14998-0.53720-1.27053-0.51562.h5
Epoch 4/50
21/21 [==============================] - 32s 2s/step - loss: 0.8778 - categorical_accuracy: 0.6786 - val_loss: 1.1770 - val_categorical_accuracy: 0.5547

Epoch 00004:

21/21 [==============================] - 32s 2s/step - loss: 0.0302 - categorical_accuracy: 0.9985 - val_loss: 1.3038 - val_categorical_accuracy: 0.5859

Epoch 00028: saving model to model_init_2020-05-2414_09_58.387065/model-00028-0.03018-0.99851-1.30376-0.58594.h5
Epoch 29/50
21/21 [==============================] - 32s 2s/step - loss: 0.0329 - categorical_accuracy: 1.0000 - val_loss: 1.2697 - val_categorical_accuracy: 0.6172

Epoch 00029: saving model to model_init_2020-05-2414_09_58.387065/model-00029-0.03286-1.00000-1.26972-0.61719.h5

Epoch 00029: ReduceLROnPlateau reducing learning rate to 4.882812731921149e-07.
Epoch 30/50
21/21 [==============================] - 32s 2s/step - loss: 0.0198 - categorical_accuracy: 1.0000 - val_loss: 1.2195 - val_categorical_accuracy: 0.6094

Epoch 00030: saving model to model_init_2020-05-2414_09_58.387065/model-00030-0.01985-1.00000-1.21954-0.60938.h5
Epoch 31/50
21/21 [==============================] - 32s 2s/step - loss: 0.0331 - categorical_

In [15]:
conv2d_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_1 (TimeDist (32, 16, 32, 32, 32)      896       
_________________________________________________________________
time_distributed_2 (TimeDist (32, 16, 32, 32, 32)      128       
_________________________________________________________________
time_distributed_3 (TimeDist (32, 16, 30, 30, 64)      18496     
_________________________________________________________________
time_distributed_4 (TimeDist (32, 16, 30, 30, 64)      256       
_________________________________________________________________
time_distributed_5 (TimeDist (32, 16, 15, 15, 64)      0         
_________________________________________________________________
time_distributed_6 (TimeDist (32, 16, 14400)           0         
_________________________________________________________________
gru_1 (GRU)                  (32, 16, 8)               345816    
__________

In [ ]:
import maptplotlib.pyplot as plt
loss_train = conv2d_model.history['acc']
loss_val = conv2d_model.history['val_acc']
epochs = range(1,51)
plt.plot(epochs, loss_train, 'g', label='Training accuracy')
plt.plot(epochs, loss_val, 'b', label='validation accuracy')
plt.title('Training and Validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

# CONV3D

In [20]:
from keras.models import Sequential, Model
from keras.layers import Dense, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation
from keras.layers.convolutional import Conv3D, MaxPooling3D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras import optimizers
from keras.layers import Dropout

#write your model here
#covolution layers
conv3d_model = Sequential()
conv3d_model.add(Conv3D(32,kernel_size=(3,3,3),activation='relu',batch_input_shape=batch_inp_shape))
conv3d_model.add(Conv3D(64,kernel_size=(3,3,3),activation='relu'))
conv3d_model.add(MaxPooling3D(pool_size=(2,2,2)))
conv3d_model.add(Conv3D(128,kernel_size=(3,3,3),kernel_regularizer='l2',activation='relu'))
conv3d_model.add(MaxPooling3D(pool_size=(2,2,2)))
conv2d_model.add(BatchNormalization())
#conv3d_model.add(Dropout(0.25))

# extract features
conv3d_model.add(Flatten())
#conv3d_model.add(BatchNormalization())
conv3d_model.add(Dense(128,activation='relu'))
conv3d_model.add(Dropout(0.5))
conv3d_model.add(Dense(Num_Classes,activation='softmax'))

In [21]:
conv3d_model.build(batch_inp_shape)
conv3d_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_3 (Conv3D)            (32, 14, 30, 30, 32)      2624      
_________________________________________________________________
conv3d_4 (Conv3D)            (32, 12, 28, 28, 64)      55360     
_________________________________________________________________
max_pooling3d_2 (MaxPooling3 (32, 6, 14, 14, 64)       0         
_________________________________________________________________
conv3d_5 (Conv3D)            (32, 4, 12, 12, 128)      221312    
_________________________________________________________________
max_pooling3d_3 (MaxPooling3 (32, 2, 6, 6, 128)        0         
_________________________________________________________________
flatten_4 (Flatten)          (32, 9216)                0         
_________________________________________________________________
dense_5 (Dense)              (32, 128)                 1179776   
__________

In [22]:
optimiser = optimizers.Adam() #write your optimizer
conv3d_model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
#print (conv3d_model.summary())

In [23]:
conv3d_model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Epoch 1/50


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:14: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:18: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


 2/21 [=>............................] - ETA: 37s - loss: 2.4368 - categorical_accuracy: 0.2188 

/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:35: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:39: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


21/21 [==============================] - 33s 2s/step - loss: 2.1268 - categorical_accuracy: 0.2292 - val_loss: 1.8671 - val_categorical_accuracy: 0.2656

Epoch 00001: saving model to model_init_2020-05-2414_09_58.387065/model-00001-2.12682-0.22917-1.86705-0.26562.h5
Epoch 2/50
21/21 [==============================] - 31s 1s/step - loss: 1.6956 - categorical_accuracy: 0.2649 - val_loss: 1.5443 - val_categorical_accuracy: 0.3672

Epoch 00002: saving model to model_init_2020-05-2414_09_58.387065/model-00002-1.69564-0.26488-1.54426-0.36719.h5
Epoch 3/50
21/21 [==============================] - 31s 1s/step - loss: 1.5181 - categorical_accuracy: 0.3512 - val_loss: 1.3897 - val_categorical_accuracy: 0.4531

Epoch 00003: saving model to model_init_2020-05-2414_09_58.387065/model-00003-1.51808-0.35119-1.38968-0.45312.h5
Epoch 4/50
21/21 [==============================] - 31s 1s/step - loss: 1.3175 - categorical_accuracy: 0.4658 - val_loss: 1.4533 - val_categorical_accuracy: 0.4219

Epoch 00004:

21/21 [==============================] - 31s 1s/step - loss: 0.4365 - categorical_accuracy: 0.8571 - val_loss: 0.8183 - val_categorical_accuracy: 0.7500

Epoch 00028: saving model to model_init_2020-05-2414_09_58.387065/model-00028-0.43650-0.85714-0.81831-0.75000.h5
Epoch 29/50
21/21 [==============================] - 31s 1s/step - loss: 0.4367 - categorical_accuracy: 0.8542 - val_loss: 0.6590 - val_categorical_accuracy: 0.7500

Epoch 00029: saving model to model_init_2020-05-2414_09_58.387065/model-00029-0.43674-0.85417-0.65903-0.75000.h5
Epoch 30/50
21/21 [==============================] - 29s 1s/step - loss: 0.4219 - categorical_accuracy: 0.8646 - val_loss: 0.9859 - val_categorical_accuracy: 0.7266

Epoch 00030: saving model to model_init_2020-05-2414_09_58.387065/model-00030-0.42193-0.86458-0.98589-0.72656.h5
Epoch 31/50
21/21 [==============================] - 32s 2s/step - loss: 0.4262 - categorical_accuracy: 0.8616 - val_loss: 0.7282 - val_categorical_accuracy: 0.7812

Epoch 000